In [2]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from IPython.display import display, Image
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import MessagesState
from dotenv import load_dotenv

load_dotenv()

c:\Users\Arun\miniconda3\envs\langgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9b302da1-63cb-4d37-96d8-6575cb022efb-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [4]:
def chat_node(state:MessagesState):
    return {"messages":llm.invoke(state["messages"])}


In [5]:
builder = StateGraph(MessagesState)

builder.add_node("chat_node",chat_node)

builder.add_edge(START,"chat_node")
builder.add_edge("chat_node",END)

graph = builder.compile()

In [6]:
messages = [
    HumanMessage(content="Hi", name="Arun"),
    AIMessage(content="Hi Arun, How can I help today?", name="Bot"),
    HumanMessage(content="Write a 300 word essay on Radioactivity", name="Arun")
]

messages

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, name='Arun'),
 AIMessage(content='Hi Arun, How can I help today?', additional_kwargs={}, response_metadata={}, name='Bot'),
 HumanMessage(content='Write a 300 word essay on Radioactivity', additional_kwargs={}, response_metadata={}, name='Arun')]

In [7]:
graph.invoke({"messages":messages})

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, name='Arun', id='1f184dca-98cb-4559-91b1-b817de12ba48'),
  AIMessage(content='Hi Arun, How can I help today?', additional_kwargs={}, response_metadata={}, name='Bot', id='0c62938c-04a7-4841-a43b-d9b2e551b0d0'),
  HumanMessage(content='Write a 300 word essay on Radioactivity', additional_kwargs={}, response_metadata={}, name='Arun', id='5b033575-bc00-4194-b58b-253a0e809395'),
  AIMessage(content="Radioactivity, the spontaneous emission of radiation from the nucleus of an unstable atom, is a fundamental phenomenon with profound implications for science, technology, and society.  This instability arises from an imbalance in the nucleus's protons and neutrons, leading to the emission of particles or energy to achieve a more stable configuration.  This process transforms the original atom into a different isotope or element, a process known as radioactive decay.\n\nSeveral types of radiation are emitted dur

### Streaming Updates Only

In [8]:
for chunk in graph.stream({"messages":messages}, stream_mode="updates"):
    chunk["chat_node"]["messages"].pretty_print()

================================== Ai Message ==================================

Radioactivity, the spontaneous emission of radiation from the nucleus of an unstable atom, is a fundamental phenomenon with profound implications for science, technology, and society.  This instability arises from an imbalance in the nucleus's composition of protons and neutrons, leading to the emission of particles or energy in an attempt to achieve a more stable configuration.  This process transforms the original atom into a different element or isotope.

Three primary types of radiation are emitted during radioactive decay: alpha, beta, and gamma radiation. Alpha particles consist of two protons and two neutrons, essentially a helium nucleus, and are relatively easily stopped by a sheet of paper. Beta particles are high-energy electrons or positrons, possessing greater penetrating power than alpha particles, requiring thicker materials like aluminum for shielding. Gamma rays are electromagnetic radiat

### Streaming Full State

In [9]:
for chunk in graph.stream({"messages":messages}, stream_mode="values"):
    for m in chunk["messages"]:
        m.pretty_print()

================================ Human Message =================================
Name: Arun

Hi
================================== Ai Message ==================================
Name: Bot

Hi Arun, How can I help today?
================================ Human Message =================================
Name: Arun

Write a 300 word essay on Radioactivity
================================ Human Message =================================
Name: Arun

Hi
================================== Ai Message ==================================
Name: Bot

Hi Arun, How can I help today?
================================ Human Message =================================
Name: Arun

Write a 300 word essay on Radioactivity
================================== Ai Message ==================================

Radioactivity, the spontaneous emission of radiation from the nucleus of an unstable atom, is a fundamental phenomenon with far-reaching implications.  This instability arises from an imbalance in the nucleus's pro

### Streaming Tokens

In [10]:
async for event in graph.astream_events({"messages":messages}, version="v2"):
    if event["event"] == "on_chat_model_stream" and event["metadata"].get("langgraph_node","") == "chat_node":
        data = event["data"]
        print(data["chunk"].content, end="|")

Radio|activity, the spontaneous emission of radiation from unstable atomic nuclei, is a fundamental phenomenon| with profound implications for science, technology, and society.  This instability arises from| an imbalance in the nucleus's protons and neutrons, driving it to achieve a more stable configuration through the emission of particles or energy.  This process, governed| by the weak nuclear force, results in the transformation of one element into another, a process known as radioactive decay.

Several types of radiation are emitted during radioactive| decay. Alpha particles, consisting of two protons and two neutrons, are relatively large and easily stopped by a sheet of paper. Beta particles, high-energy electrons or positrons, are more penetrating and require thicker shielding, such as aluminum.| Gamma rays, high-energy electromagnetic radiation, are the most penetrating and require substantial shielding, like lead or concrete.  The type and energy of radiation emitted are char